Code for the preprocessing of the 24 hours evaluation files in preparation for the HRV analysis

**Instalation of the packages**

In [ ]:
pip install "devicely"
pip install "jointly"

**Import necessary packages**

In [ ]:
import devicely
import jointly
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

## Read Empatica data ##

In [ ]:
empatica_reader = devicely.EmpaticaReader("path") 
#read the Empatica data from subject 

In [ ]:
empatica_reader.data.head() #check the head of the whole dataset

In [ ]:
empatica_reader.IBI.head() #check the head of Empatica's IBI

**Create a subset with only IBI data (EMPATICA)**

In [ ]:
empatica_test = empatica_reader # copy the data into a working subset

In [ ]:
empatica_test.data.head()  #check and match the head of the data

In [ ]:
#move the date index to align with Faros
utc_shift =  pd.Timedelta('-15 day')
empatica_test.timeshift(utc_shift)

empatica_test.data.head()

In [ ]:
# move the time index to align with Faros
utc_shift1 =  pd.Timedelta('+2 hours')
empatica_test.timeshift(utc_shift1)

empatica_test.data.head()

In [ ]:
empatica_test = empatica_test.data.reset_index() # delete the multiIndex

In [ ]:
empatica_test.head()

In [ ]:
 #delete the unnecessary columns
empatica_test = empatica_test.drop(columns = ["bvp", "eda", "hr", "temp", "acc_x", "acc_y", "acc_z", "acc_mag", "timedelta"])

In [ ]:
empatica_test.rename (columns = {"index":"time"}, inplace= True)

In [ ]:
empatica_test #check the data

In [ ]:
empatica_test = empatica_test.loc[empatica_test["ibi"].isnull() != True] # drop all the NaN rows

In [ ]:
 #drop the values of RR = 0 (an interval between to subsequents RR can not be 0)
empatica_test = empatica_test.drop(empatica_test[empatica_test.ibi == 0.0].index)

In [ ]:
 #save the working subset
empatica_test.to_csv("path...\empatica.csv", date_format = "%Y-%m-%d %H:%M:%S.%f", index=False)

## Read Faros data ##

In [ ]:
faros_reader = devicely.FarosReader("path")
#read the Faros data from subject 

In [ ]:
faros_reader.data #check the head of the whole dataset

**Create a subset with only IBI data (FAROS)**

In [ ]:
faros_test = faros_reader # copy the data into a working subset

In [ ]:
faros_test.data.head() #check and match the head of the data

In [ ]:
faros_test = faros_test.data.reset_index() # delete the multiIndex

In [ ]:
faros_test.head()

In [ ]:
 #delete the unnecessary columns
faros_test = faros_test.drop(columns = ["ECG", "Accelerometer_X", "Accelerometer_Y", "Accelerometer_Z", "Marker", "acc_mag"])

In [ ]:
faros_test #check the data

In [ ]:
faros_test = faros_test.loc[faros_test["HRV"].isnull() != True] # drop all the NaN rows

In [ ]:
faros_test = faros_test.drop(faros_test[faros_test.HRV == 0.0].index) #drop the values of RR = 0 (an interval between to subsequents RR can not be 0)

In [ ]:
 #save the working subset
faros_test.to_csv("path...\faros.csv", date_format = "%Y-%m-%d %H:%M:%S.%f", index=False)

## Call our new working data sets

In [ ]:
empatica_short = pd.read_csv("path...\empatica.csv")

In [ ]:
empatica_short

In [ ]:
faros_short = pd.read_csv("path...\faros.csv")

In [ ]:
faros_short

### Set the indeces for the data-reading

In [ ]:
empatica_short["time"]= pd.to_datetime(empatica_short['time']) # transform the time into data time.

In [ ]:
empatica_short.dtypes #check the types of the data, proff time is a correct type

In [ ]:
empatica_short.head() #check the head(content) of the table. It is important to notice that for the different sample rates, 

In [ ]:
empatica_short = empatica_short.set_index('time')

In [ ]:
empatica_short["ibi"] = empatica_short["ibi"]*1000 # the scale of the empatica data is fixed

In [ ]:
faros_short["time"]= pd.to_datetime(faros_short['time']) # transform the time into data time.

In [ ]:
faros_short.dtypes

In [ ]:
faros_short.head()

In [ ]:
faros_short = faros_short.set_index('time')

## Cut and align DATA ##

In [ ]:
#we cut the data to the self-reported sleep time example "23:30:00","06:30:00"
empatica_short = empatica_short.between_time("start_time","end_time")

In [ ]:
#save the filtered subset into a new file
empatica_short.to_csv("path...\RR_empatica_fit.csv") 

In [ ]:
#we cut the data to the self-reported sleep time example "23:30:00","06:30:00"
faros_short = faros_short.between_time("start_time","end_time")

In [ ]:
#save the filtered subset into a new file
faros_short.to_csv("path...\RR_faros_fit.csv") 

## Plot

In [ ]:
merged_data = pd.merge(empatica_short, faros_short, how="outer", on="time")
merged_data.rename(columns={"ibi": "Empatica", "HRV": "Faros"}, inplace = True)

In [ ]:
plot = merged_data.interpolate(method='time').plot(figsize=(15,6), ylim=(300,1500))

## Cut in 10 mins segments and create files from each one

In [ ]:
start_time = empatica_short.index[0] #define our initial time (PPG device)
end = empatica_short.index[-1] - timedelta(minutes = 10) #end time limit of the analyzed data
i = 0
while start_time <= end:
    
    last_time = start_time + timedelta(minutes = 10)
    df = empatica_short.between_time((pd.to_datetime(start_time).strftime("%H:%M:%S")), (pd.to_datetime(last_time)).strftime("%H:%M:%S"))
    #df holding the 10 minutes segment from slicing the original data
    df.to_csv("path...\Empatica_Segment{}.csv".format(i)) 
    start_time = start_time + timedelta(minutes = 5)
    i = i + 1

In [ ]:
start_time = faros_short.index[0] #define our initial time (ECG device)
end = faros_short.index[-1] - timedelta(minutes = 10) #end time limit of the analyzed data
i = 0
while start_time <= end:
    
    last_time = start_time + timedelta(minutes = 10)
    df = faros_short.between_time((pd.to_datetime(start_time).strftime("%H:%M:%S")), (pd.to_datetime(last_time)).strftime("%H:%M:%S"))
    #df holding the 10 minutes segment from slicing the original data
    df.to_csv("path...\Faros_Segment{}.csv".format(i)) 
    start_time = start_time + timedelta(minutes = 5)
    i = i + 1